# Extract, Transform, Load

# 1. Libraries

In [1]:
#data organizing
import pandas #storage
import numpy as np #data-type conversion
from os import getcwd

#splitting the dataset - simple method
from sklearn.model_selection import train_test_split

# 2. Dataset - Importing

In [2]:
def cvSex(sex):
    mapper = {1: 'Male', 2: 'Female'}
    return mapper.get(sex)

def cvEducation(education):
    mapper = {1: 'Graduate', 2: 'University', 3: 'High School', 4: 'Others'}
    return mapper.get(education, 'Others') #takes care of cases 0, 5 and 6

def cvMarriage(marriage):
    mapper = {1: 'Married', 2: 'Single', 3: 'Divorced', 4: 'Others'}
    return mapper.get(marriage, 'Others') #takes care of 54 entries

def cvPayHistory(payHistory):
    mapper = {-2: 'No Credit Use', -1: 'Paid in Full', 0: 'Revolving Credit Used', 1: 'Delay 1 of month', 2: 'Delay 2 of months', 3: 'Delay 3 of months', 4: 'Delay 4 of months', 5: 'Delay 5 of months', 6: 'Delay 6 of months', 7: 'Delay 7 of months', 8: 'Delay 8 of months', 9: 'Delay 9 of months OR more'}
    return mapper.get(payHistory)


def cvDefPay(prediction):
    mapper = {0: False, 1: True}
    return mapper.get(prediction)

In [3]:
#BILL_AMT1 to PAY_AMT6 should NOT have 0 replaced with NAN
#dtype changed from int64 to int32 to save space and speed up computation, no data was lost
url = getcwd() + '\\default of credit card clients.xls'
ccd = pandas.read_excel(io = url, \
                        sheet_name='Data', header = 1, index_col = 0, \
                        dtype = {'LIMIT_BAL': np.int32, 'AGE': np.int32, 'BILL_AMT1': np.int32, 'BILL_AMT2': np.int32, 'BILL_AMT3': np.int32, 'BILL_AMT4': np.int32, 'BILL_AMT5': np.int32, 'BILL_AMT6': np.int32, 'PAY_AMT1': np.int32, 'PAY_AMT2': np.int32, 'PAY_AMT3': np.int32, 'PAY_AMT4': np.int32, 'PAY_AMT5': np.int32, 'PAY_AMT6': np.int32}, \
                        converters = {'SEX': cvSex, 'EDUCATION': cvEducation, 'MARRIAGE': cvMarriage, 'default payment next month': cvDefPay, 'PAY_0': cvPayHistory, 'PAY_2': cvPayHistory, 'PAY_3': cvPayHistory, 'PAY_4': cvPayHistory, 'PAY_5': cvPayHistory, 'PAY_6': cvPayHistory,})

In [4]:
ccd.rename(columns = {'PAY_0': 'PAY_1'}, inplace = True)
ccd.rename(columns = {'default payment next month': 'default_payment_next_month'}, inplace = True)

# 3. Feature Engineering

## 3.1. PAY {PAY_1 to PAY_6}

1. Using mode to aggregate. An entry may have mutiple mode values (same frequency), to resolve, using severest class.

2. Why severest value? To ensure fiscally fit population of credit users.

In [5]:
ccdr = pandas.read_excel(io = url, 
                        sheet_name='Data', header = 1, index_col = 0)
ccdr.rename(columns = {'PAY_0': 'PAY_1'}, inplace = True)

In [6]:
ccdrHistory = ccdr[['PAY_1', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6']]

In [7]:
ccdrHistoryMode = ccdrHistory.mode(axis = 'columns')
ccdrHistorySeverest = ccdrHistoryMode.apply(func = max, axis = 'columns')
ccdPayHistoryMode = map(cvPayHistory, ccdrHistorySeverest)

In [8]:
ccd['PAY_MODE_SEVEREST'] = list(ccdPayHistoryMode)

## 3.2. BILL_AMT {BILL_AMT1 to BILL_AMT6}

Using mean for total credit used

In [9]:
ccdSpent = ccd[['BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6']]

In [10]:
ccd['BILL_AMT_MEAN'] = np.int32(ccdSpent.mean(axis = 'columns').round())

## 3.3. PAY_AMT {PAY_AMT1 to PAY_AMT6}

Using mean for total credit settled

In [11]:
ccdSettled = ccd[['PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']]

In [12]:
ccd['PAY_AMT_MEAN'] = np.int32(ccdSettled.mean(axis = 'columns').round())

# 4. Splitting Dataset

In [13]:
ccdY = ccd['default_payment_next_month']
ccdX = ccd.drop(['default_payment_next_month'], axis = 'columns')

In [14]:
trainX, testX, trainY, testY = train_test_split(ccdX, ccdY, test_size = 0.25, random_state = 44)